In [1]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import pairwise
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

from imblearn.over_sampling import RandomOverSampler

from dmba import classificationSummary
from dmba import plotDecisionTree, classificationSummary, gainsChart, liftChart

%matplotlib inline

In [9]:
absent = pd.read_csv('/Users/mtc/ADS/ADS 505/Project/Absenteeism_at_work.csv', delimiter = ";")

In [10]:
absent.head()

,ID,Reason for absence,Month of absence,Day of the week,Seasons,Transportation expense,Distance from Residence to Work,Service time,Age,Work load Average/day,...,Disciplinary failure,Education,Son,Social drinker,Social smoker,Pet,Weight,Height,Body mass index,Absenteeism time in hours
0,11,26,7,3,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,4
1,36,0,7,3,1,118,13,18,50,239.554,...,1,1,1,1,0,0,98,178,31,0
2,3,23,7,4,1,179,51,18,38,239.554,...,0,1,0,1,0,0,89,170,31,2
3,7,7,7,5,1,279,5,14,39,239.554,...,0,1,2,1,1,0,68,168,24,4
4,11,23,7,5,1,289,36,13,33,239.554,...,0,1,2,1,0,1,90,172,30,2


In [12]:
count = pd.DataFrame(absent['ID'].value_counts())

count = count.sort_values('ID')

print(count) #IDs 8, 4, and 35 have no hours of absence, but this will be taken care of later, since measures will be in hours/unit of time

    count
ID       
1      23
2       6
3     113
4       1
5      19
6       8
7       6
8       2
9       8
10     24
11     40
12      7
13     15
14     29
15     37
16      2
17     20
18     16
19      3
20     42
21      3
22     46
23      8
24     30
25     10
26      5
27      7
28     76
29      5
30      7
31      3
32      5
33     24
34     55
35      1
36     34


In [13]:
count_reason = pd.DataFrame(absent['Reason for absence'].value_counts())

count_reason = count_reason.sort_values('count', ascending=True)

print(count_reason)

                    count
Reason for absence       
17                      1
3                       1
2                       1
4                       2
15                      2
24                      3
16                      3
5                       3
9                       4
8                       6
21                      6
12                      8
6                       8
7                      15
1                      16
14                     19
18                     21
10                     25
11                     26
25                     31
26                     33
22                     38
19                     40
0                      43
13                     55
27                     69
28                    112
23                    149


In [15]:
indv_info = pd.DataFrame(absent.groupby(['ID'])['Absenteeism time in hours'].sum())

indv_info['abs_hr_per_month'] = indv_info['Absenteeism time in hours']/36

indv_info['abs_hr_per_log'] = indv_info['Absenteeism time in hours']/count['count']

In [7]:
print(indv_info.sort_values('abs_hr_per_log', ascending=False))

    Absenteeism time in hours  abs_hr_per_month  abs_hr_per_log
ID                                                             
9                         262          7.277778       32.750000
26                         83          2.305556       16.600000
14                        476         13.222222       16.413793
13                        183          5.083333       12.200000
11                        450         12.500000       11.250000
36                        311          8.638889        9.147059
6                          72          2.000000        9.000000
24                        254          7.055556        8.466667
16                         16          0.444444        8.000000
10                        186          5.166667        7.750000
18                        118          3.277778        7.375000
20                        306          8.500000        7.285714
15                        253          7.027778        6.837838
17                        126          3

In [14]:
print(indv_info.sort_values('abs_hr_per_month', ascending=False))

    Absenteeism time in hours  abs_hr_per_month  abs_hr_per_log
ID                                                             
3                         482         13.388889        4.265487
14                        476         13.222222       16.413793
11                        450         12.500000       11.250000
28                        347          9.638889        4.565789
34                        344          9.555556        6.254545
36                        311          8.638889        9.147059
20                        306          8.500000        7.285714
9                         262          7.277778       32.750000
24                        254          7.055556        8.466667
15                        253          7.027778        6.837838
22                        253          7.027778        5.500000
10                        186          5.166667        7.750000
13                        183          5.083333       12.200000
17                        126          3

In [22]:
print(indv_info.sort_values('Absenteeism time in hours', ascending=False))

    Absenteeism time in hours  abs_hr_per_month  abs_hr_per_log
ID                                                             
3                         482         13.388889        4.265487
14                        476         13.222222       16.413793
11                        450         12.500000       11.250000
28                        347          9.638889        4.565789
34                        344          9.555556        6.254545
36                        311          8.638889        9.147059
20                        306          8.500000        7.285714
9                         262          7.277778       32.750000
24                        254          7.055556        8.466667
15                        253          7.027778        6.837838
22                        253          7.027778        5.500000
10                        186          5.166667        7.750000
13                        183          5.083333       12.200000
17                        126          3

In [25]:
print(pd.DataFrame(absent[(absent['ID'] == 9)].value_counts('Reason for absence')))

                    count
Reason for absence       
6                       2
18                      2
25                      2
1                       1
12                      1


In [26]:
print(pd.DataFrame(absent[(absent['ID'] == 3)].value_counts('Reason for absence')))

                    count
Reason for absence       
27                     38
28                     26
23                     19
13                     10
11                      7
10                      2
18                      2
21                      2
25                      2
0                       1
5                       1
6                       1
12                      1
26                      1
